In [29]:
import logging
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sqlalchemy import exc, create_engine
import pymysql
import pandas as pd

logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [2]:
try:
    conn = pymysql.connect(host=os.environ.get('IP'), user=os.environ.get('RDS_USER'), passwd=os.environ.get('RDS_PW'),
     db=os.environ.get('jacob_db'), connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    sys.exit()


In [24]:
def get_player_stats():
    year = 2021
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]

    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

    stats = DataFrame(player_stats, columns = headers)
    stats['PTS'] = to_numeric(stats['PTS'])
    logger.info(f'General Stats Function Successful, retrieving {len(stats)} updated rows')
    return(stats)

In [25]:
stats = get_player_stats()

INFO:root:General Stats Function Successful, retrieving 731 updated rows


In [6]:
connection = create_engine('mysql+mysqlconnector://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + os.environ.get('PORT') + '/' + os.environ.get('RDS_DB'),
                     echo = False)

ModuleNotFoundError: No module named 'mysql'

In [15]:
stats.to_sql(con = connection, name = "player_stats", index = False, if_exists = "replace")

In [10]:
all_tables1 = pd.read_sql_query('SHOW TABLES FROM jacob_db;', connection)

In [17]:
all_tables1

,Tables_in_jacob_db
0,aws_odds_df
1,my_python_injuries
2,my_python_table
3,player_stats


In [26]:
def sql_connection():
    try:
        connection = create_engine('mysql+pymysql://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + os.environ.get('PORT') + '/' + os.environ.get('RDS_DB'),
                     echo = False)
        logging.info('SQL Connection Successful')
        return(connection)
    except exc.SQLAlchemyError as e:
        logging.error('SQL Connection Failed, Error:', e)
        return e


In [27]:
engine = sql_connection()

INFO:root:SQL Connection Successful


In [33]:
all_tables1 = pd.read_sql_query('SHOW TABLES FROM jacob_db;', engine)

In [35]:
all_tables1

,Tables_in_jacob_db
0,aws_odds_df
1,my_python_injuries
2,my_python_table
3,player_stats
4,player_stats2
5,player_stats3
6,player_stats44


In [28]:
stats.to_sql(con = engine, name = "player_stats44", index = False, if_exists = "replace")